In [1]:
pip install pyEDFlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
# libraries installation
from pyedflib import EdfReader
from numpy import zeros, arange
import csv
import pandas as pd
from csv import reader, writer
import os

In [4]:
#paths defines
Edfpath='edf_files/'
tempath='temp/'

# read the excel file made for patient data
excelData=pd.read_csv('Siena DB timestamp.csv', header=None)
total_no_of_edfs=50
excelData

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,Patient,Sub File,No of channels,Channels not used,Seizure,Localization,Lateralization,No of Seizure,Registeration Time,Seizure Start Time,...,Post-Ictal Start Row,Post Ictal end Row,Pre Ictal Start Time,Pre Ictal End Time,Pre Ictal Start Row,Pre Ictal End Row,Peri Ictal start time,Peri Ictal End Time,Peri Ictal start Row,Peri Ictal End Row
1,0,PN00-1,34,NaN,IAS,T,R,1,70773,71916,...,623616,659456,1068,1138,546816,582656,1138,1218,582656,623616
2,0,PN00-2,34,NaN,IAS,T,R,1,8297,9517,...,654848,682496,1161,1215,594432,622080,1215,1279,622080,654848
3,0,PN00-4,34,NaN,IAS,T,R,1,75103,76109,...,555520,593408,927,1001,474624,512512,1001,1085,512512,555520
4,0,PN00-5,34,NaN,IAS,T,R,1,80524,81428,...,499712,534016,832,899,425984,460288,899,976,460288,499712
5,1,PN01-1_1,34,NaN,IAS,T,L,2,68444,78662,...,5261824,5289472,10159,10213,5201408,5229056,10213,10277,5229056,5261824
6,1,PN01-1_2,34,NaN,IAS,T,L,2,68444,114797,...,23773184,23811072,46274,46348,23692288,23730176,46348,46432,23730176,23773184
7,3,PN03-1,34,NaN,IAS,T,R,1,81877,120550,...,19859968,19916800,38557,38668,19741184,19798016,38668,38789,19798016,19859968
8,3,PN03-2,34,NaN,IAS,T,R,1,77464,112385,...,17950208,18018304,34783,34916,17808896,17876992,34916,35059,17876992,17950208
9,5,PN05-2,34,NaN,IAS,T,L,1,24362,31525,...,3687936,3705856,7123,7158,3646976,3664896,7158,7203,3664896,3687936


**ICTAL**

In [ ]:
seizuren= 0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
  seizure_type= excelData[4][i]
  localization=excelData[5][i]
  lateralization=excelData[6][i]


# read Ictal start and end rows
  IctalS= int(excelData[15][i])
  IctalE= int(excelData[16][i])

  print(seizure_type,localization,lateralization,IctalS,IctalE)

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

  #store all data
  data_eeg = data

  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)

  # convert the edf to csv file
  df.to_csv(tempath+'temp1.csv', index=False)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for ictal
  cols=list(range(0,IctalS))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(IctalE+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=1

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create pateint specific folder
  folderpath = 'Generated siena db/Patient Inter Specific/'+seizure_type+'/'+localization+'/'+lateralization+'/'+'PN'+str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)

  newpath= folderpath + 'Ictal' +'/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
  print(df.shape)
  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + subfile +'_ictal.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))
  df=pd.read_csv(newpath + subfile +'_ictal.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)
  print(df.shape)

  # Patient Non Specific
  newpath2= 'Generated siena db/Patient Non-Specific/Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)

  df.to_csv(newpath + subfile +'_ictal.csv', index=False, header=False)
  df.to_csv(newpath2+str(i+seizuren)+'.csv', index=False, header=False)

**POST-ICTAL**

In [ ]:
seizuren= 0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
  seizure_type= excelData[4][i]
  localization=excelData[5][i]
  lateralization=excelData[6][i]


# read Post Ictal start and end rows
  PostIctalS= int(excelData[19][i])
  PostIctalE= int(excelData[20][i])

  print(seizure_type,localization,lateralization,PostIctalS,PostIctalE)

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

  #store all data
  data_eeg = data

  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)

  # convert the edf to csv file
  df.to_csv(tempath+'temp1.csv', index=False)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for post-ictal
  cols=list(range(0,PostIctalS))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(PostIctalE+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=1

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create pateint specific folder
  folderpath = 'Generated siena db/Patient Inter Specific/'+seizure_type+'/'+localization+'/'+lateralization+'/'+'PN'+str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)
  newpath= folderpath + 'Post-Ictal' +'/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
  print(df.shape)
  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + subfile +'_post-ictal.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))
  df=pd.read_csv(newpath + subfile +'_post-ictal.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)
  print(df.shape)

  
  # Patient Non Specific
  newpath2= 'Generated siena db/Patient Non-Specific/Post-Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)


  df.to_csv(newpath + subfile +'_post-ictal.csv', index=False, header=False)
  df.to_csv(newpath2+str(i+seizuren)+'.csv', index=False, header=False)

**PRE_ICTAL**

In [ ]:
seizuren= 0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
  seizure_type= excelData[4][i]
  localization=excelData[5][i]
  lateralization=excelData[6][i]


# read Pre Ictal start and end rows
  PreIctalS= int(excelData[23][i])
  PreIctalE= int(excelData[24][i])

  print(seizure_type,localization,lateralization,PreIctalS,PreIctalE)

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

  #store all data
  data_eeg = data

  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)

  # convert the edf to csv file
  df.to_csv(tempath+'temp1.csv', index=False)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for pre-ictal
  cols=list(range(0,PreIctalS))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(PreIctalE+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=1

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create pateint specific folder
  folderpath = 'Generated siena db/Patient Inter Specific/'+seizure_type+'/'+localization+'/'+lateralization+'/'+'PN'+str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)
  newpath= folderpath + 'Pre-Ictal' +'/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
  print(df.shape)
  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + subfile +'_pre-ictal.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))
  df=pd.read_csv(newpath + subfile +'_pre-ictal.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)
  print(df.shape)

  
  # Patient Non Specific
  newpath2= 'Generated siena db/Patient Non-Specific/Pre-Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)


  df.to_csv(newpath + subfile +'_pre-ictal.csv', index=False, header=False)
  df.to_csv(newpath2+str(i+seizuren)+'.csv', index=False, header=False)

**PERI_ICTAL**

In [ ]:
seizuren= 0
for i in range(1,total_no_of_edfs):

  edfFile=excelData[1][i]
  patientno= excelData[0][i]
  subfile=excelData[1][i]
  seizure_type= excelData[4][i]
  localization=excelData[5][i]
  lateralization=excelData[6][i]


# read Peri Ictal start and end rows
  PeriIctalS= int(excelData[27][i])
  PeriIctalE= int(excelData[28][i])

  print(seizure_type,localization,lateralization,PeriIctalS,PeriIctalE)

# read edf file
  f=EdfReader(Edfpath+ edfFile+'.edf')
  n = f.signals_in_file
  signal_labels = f.getSignalLabels()
  print(signal_labels)
  data = zeros((n,f.getNSamples()[0]))
  data
  for k in arange(n):
      data[k, :] = f.readSignal(k)

  #store all data
  data_eeg = data

  #close the edf file
  f._close()
  del f
  print(data_eeg.shape)

  df = pd.DataFrame(data_eeg)

  # convert the edf to csv file
  df.to_csv(tempath+'temp1.csv', index=False)
  total_cols= len(df.columns)
  print(total_cols)

  #cut the data for peri-ictal
  cols=list(range(0,PeriIctalS))
  df.drop(columns=cols, axis=1,inplace=True)
  cols=list(range(PeriIctalE+1,total_cols))
  df.drop(columns=cols, axis=1,inplace=True)
  print(df.shape)

  # add class label
  df[len(df.columns)]=1

  #save the updated csv file
  df.to_csv(tempath+'temp2.csv', index=False)


  # create pateint specific folder
  folderpath = 'Generated siena db/Patient Inter Specific/'+seizure_type+'/'+localization+'/'+lateralization+'/'+'PN'+str(patientno)+'/'
  if not os.path.exists(folderpath):
    os.makedirs(folderpath)
  newpath= folderpath + 'Peri-Ictal' +'/'
  if not os.path.exists(newpath):
    os.makedirs(newpath)
  print(df.shape)
  # transpose the csv
  with open(tempath+'temp2.csv') as f, open(newpath + subfile +'_peri-ictal.csv', 'w') as fw:
      writer(fw, delimiter=',').writerows(zip(*reader(f, delimiter=',')))
  df=pd.read_csv(newpath + subfile +'_peri-ictal.csv', header=None)
  df.drop(columns=0, axis=1,inplace=True)
  print(df.shape)

  
  # Patient Non Specific
  newpath2= 'Generated siena db/Patient Non-Specific/Peri-Ictal/'
  if not os.path.exists(newpath2):
    os.makedirs(newpath2)


  df.to_csv(newpath + subfile +'_peri-ictal.csv', index=False, header=False)
  df.to_csv(newpath2+str(i+seizuren)+'.csv', index=False, header=False)